In [8]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from keras.layers import Dense, Input, Dropout
from keras import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [9]:
from keras.utils.np_utils import to_categorical

X_train = pd.read_csv(r"D:\Northeastern Semester 1\Projects\ml_project\Data_Set\x_train_encode.csv")
X_test = pd.read_csv(r"D:\Northeastern Semester 1\Projects\ml_project\Data_Set\x_test_encode.csv")
y_train = pd.read_csv(r"D:\Northeastern Semester 1\Projects\ml_project\Data_Set\y_train_encode.csv")
y_test = pd.read_csv(r"D:\Northeastern Semester 1\Projects\ml_project\Data_Set\y_test_encode.csv")

X_train.drop(columns = ['Unnamed: 0'], axis =1, inplace = True)
X_test.drop(columns = ['Unnamed: 0'], axis = 1, inplace = True)

""" y_train = to_categorical(y_train)
y_test =  to_categorical(y_test) """
y_test.head()

,Y
0,0
1,1
2,0
3,1
4,0


In [10]:
'''Data Cleaning'''
df = pd.read_csv(r"D:\Northeastern Semester 1\Projects\ml_project\Data_Set\Vehicle_Coupon.csv")
df.drop(['car', 'direction_same', 'toCoupon_GEQ5min'], axis=1, inplace=True)
df['temperature'] = df['temperature'].astype(str)
df = df.apply(lambda x: x.fillna(x.value_counts().index[0]))

df_ohe = pd.get_dummies(df)
X, y = df_ohe.drop(['Y'], axis=1), df_ohe['Y']

X_trainn, X_testt, y_trainn, y_testt = train_test_split(X, y, test_size=0.3, random_state=42)
X_trainn, X_validd, y_trainn, y_validd = train_test_split(X_trainn, y_trainn, test_size=0.2, random_state=42)

In [11]:
def algorithm_pipeline(X_train_data, X_test_data, y_train_data, y_test_data, 
                       model, param_grid, cv, scoring,
                       do_probabilities = False):
    gs = GridSearchCV(
        estimator=model,
        param_grid=param_grid, 
        cv=cv, 
        n_jobs=-1, 
        scoring=scoring,
        verbose=2
    )
    fitted_model = gs.fit(X_train_data, y_train_data)

    pred = gs.predict(X_test_data, y_test_data)

    return fitted_model, pred

In [22]:
def modell(activation, dropout,
              optimizer):
    
    
    model = Sequential()
    
    model.add(Dense(200,activation))
    model.add(Dropout(dropout))
    model.add(Dense(60,activation))
    model.add(Dropout(dropout))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer=optimizer,
    metrics=['accuracy'])
    
    return model

tf = modell(activation= 'relu', optimizer= 'Adam', dropout= 0.2)

tf.fit(X_trainn, y_trainn, batch_size= 128, epochs= 200)

tf.evaluate(X_testt, y_testt)

Epoch 1/200
56/56 [==============================] - 1s 3ms/step - loss: 0.6492 - accuracy: 0.6215
Epoch 2/200
56/56 [==============================] - 0s 3ms/step - loss: 0.6011 - accuracy: 0.6792
Epoch 3/200
56/56 [==============================] - 0s 3ms/step - loss: 0.5832 - accuracy: 0.6980
Epoch 4/200
56/56 [==============================] - 0s 3ms/step - loss: 0.5653 - accuracy: 0.7152
Epoch 5/200
56/56 [==============================] - 0s 3ms/step - loss: 0.5450 - accuracy: 0.7271
Epoch 6/200
56/56 [==============================] - 0s 3ms/step - loss: 0.5194 - accuracy: 0.7451
Epoch 7/200
56/56 [==============================] - 0s 3ms/step - loss: 0.4982 - accuracy: 0.7616
Epoch 8/200
56/56 [==============================] - 0s 3ms/step - loss: 0.4800 - accuracy: 0.7688
Epoch 9/200
56/56 [==============================] - 0s 3ms/step - loss: 0.4579 - accuracy: 0.7848
Epoch 10/200
56/56 [==============================] - 0s 3ms/step - loss: 0.4386 - accuracy: 0.8001
Epoch 11/

[1.5064598321914673, 0.7385706901550293]

In [17]:
tf.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 200)               22000     
                                                                 
 dropout_10 (Dropout)        (None, 200)               0         
                                                                 
 dense_16 (Dense)            (None, 60)                12060     
                                                                 
 dropout_11 (Dropout)        (None, 60)                0         
                                                                 
 dense_17 (Dense)            (None, 1)                 61        
                                                                 
Total params: 34,121
Trainable params: 34,121
Non-trainable params: 0
_________________________________________________________________


In [5]:
param_grid = {
              'epochs':[100, 200, 200, 500],
              'batch_size':[64, 128, 256],
              'optimizer' :['Adam', 'Nadam'],
              'dropout' : [0.2,0.3, 0.5],
              'activation' : ['relu', 'elu']
             }

model = KerasClassifier(build_fn = modell, verbose=0)

model, pred = algorithm_pipeline(X_train, X_test, y_train, y_test, model, 
                                        param_grid, cv=10, scoring='accuracy')

print(model.best_score_)
print(model.best_params_)

Fitting 10 folds for each of 144 candidates, totalling 1440 fits


c:\Users\Hemant\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  if __name__ == '__main__':
